In [1]:
import json
import pickle
from tqdm import tqdm
from spacy.lang.hi import Hindi
from spacy.lang.en import English
from collections import Counter
import pickle

2023-10-13 19:34:21.165471: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
class Kun:
    def __init__(self):
        self.maps={}
        self.docs={}
    
    def update(self,k,v):
        if k not in self.maps:
            self.maps[k]=Counter()
            self.docs[k]=0
        self.maps[k].update(set(v))
        self.docs[k]= self.docs[k]+1

In [9]:
suffixes_map = {
        1: ["_OH","_EH","_OO","_U","_EE","_I","_A"],
        2: ["_KAR","_AAO","_IYE","_AAI","_AAE","_NE",u"_NEE","_NAA","_TE","_EEN","_TEE","_TAA","_AANG","_AAN","_ONN","_EN"],
        3: ["_AAKAR","_AAIYE","_AAEEN","_AAYAA","_EGEE","_EGAA","_OGEE","_OGE","_AANE","_AANAA","_AATE","_AATEE","_AATAA","_TEEN","_AAON","_AAEN","_UON","_UEN","_UAAN"],
        4: ["_AAEGI","_AAEGA","_AAOGI","_AAOGE","_AENGI","_ENGI","_AENGE","_ENGE","_OONGI","_OONGA","_AATEEN",u"_NAAON",u"_NAAEN",u"_TAAON",u"_TAAEN",u"_IYAANC",u"_IYON",u"_IYAAN"],
        5: ["_AAENGEE","_AAENGE","_AAOUNGEE","_AAOUNGAA","_AAIYAAN","_AAIYON","_AAIYAANN"],
    }

In [10]:
def generate_stem_words(word,buffer=1):
    suffixes = {
        1: [u"ो",u"े",u"ू",u"ु",u"ी",u"ि",u"ा"],
        2: [u"कर",u"ाओ",u"िए",u"ाई",u"ाए",u"ने",u"नी",u"ना",u"ते",u"ीं",u"ती",u"ता",u"ाँ",u"ां",u"ों",u"ें"],
        3: [u"ाकर",u"ाइए",u"ाईं",u"ाया",u"ेगी",u"ेगा",u"ोगी",u"ोगे",u"ाने",u"ाना",u"ाते",u"ाती",u"ाता",u"तीं",u"ाओं",u"ाएं",u"ुओं",u"ुएं",u"ुआं"],
        4: [u"ाएगी",u"ाएगा",u"ाओगी",u"ाओगे",u"एंगी",u"ेंगी",u"एंगे",u"ेंगे",u"ूंगी",u"ूंगा",u"ातीं",u"नाओं",u"नाएं",u"ताओं",u"ताएं",u"ियाँ",u"ियों",u"ियां"],
        5: [u"ाएंगी",u"ाएंगे",u"ाऊंगी",u"ाऊंगा",u"ाइयाँ",u"ाइयों",u"ाइयां"],
    }
    
    for L in suffixes:
        if len(word) > L + buffer:
            for suf in suffixes[L]:
                #print type(suf),type(word),word,suf
                if word.endswith(suf):
                    #print 'h'
                    return word[:-L]
    return word

mcw_map=[
 ('।', '.'),
 ('और', 'and'),
 (',', ','),
 ('"', '"'),
 ('.', '.'),
 ('!', '!'),
 ('-', '-')
]

In [11]:
nlp_en = English()
nlp_hi = Hindi()

tok_hi = nlp_hi.tokenizer
tok_en = nlp_en.tokenizer

nlp_en.add_pipe('sentencizer')
nlp_hi.add_pipe('sentencizer')

In [12]:
vocab_hi=pickle.load(open("./models/vocab_hi.bin","rb"))
vocab_en=pickle.load(open("./models/vocab_en.bin","rb"))

eng_stories=json.load(open("/Users/ppuser/kllama/tinystories/stories_unique_50.json"))
hin_stories=json.load(open("/Users/ppuser/kllama/hinllama/data/hindi_dataset/stories.json","r"))

kunfile_hi=pickle.load(open("kunfile_hi.bin","rb"))
kunfile_en=pickle.load(open("kunfile_en.bin","rb"))

print(f"""
English vocabulary size: {len(vocab_en)}
Hindi vocabulary size: {len(vocab_hi)}
""")


English vocabulary size: 51225
Hindi vocabulary size: 22934



In [13]:
len(eng_stories)

41346

In [15]:
stories_idx=pickle.load(open("consistent_stories.bin","rb"))
final_mapping=json.load(open("final_mapping.json","r"))
stage4=json.load(open("stage4.json","r"))
for k in mcw_map:
    final_mapping[k[0]]=k[1]
stem=json.load(open("stem.json"))
eng_words=set([x[1] for x in final_mapping.items()])
hi_words=set([x[0] for x in final_mapping.items()])

In [17]:
hi_to_en_map=final_mapping.copy()
for k,v in stage4.items():
    hi_to_en_map[k]=v

In [18]:
len(hi_to_en_map)

15422

In [32]:
def stem_tokenize(word,buffer=1):
    suffixes = {
        1: [u"ो",u"े",u"ू",u"ु",u"ी",u"ि",u"ा"],
        2: [u"कर",u"ाओ",u"िए",u"ाई",u"ाए",u"ने",u"नी",u"ना",u"ते",u"ीं",u"ती",u"ता",u"ाँ",u"ां",u"ों",u"ें"],
        3: [u"ाकर",u"ाइए",u"ाईं",u"ाया",u"ेगी",u"ेगा",u"ोगी",u"ोगे",u"ाने",u"ाना",u"ाते",u"ाती",u"ाता",u"तीं",u"ाओं",u"ाएं",u"ुओं",u"ुएं",u"ुआं"],
        4: [u"ाएगी",u"ाएगा",u"ाओगी",u"ाओगे",u"एंगी",u"ेंगी",u"एंगे",u"ेंगे",u"ूंगी",u"ूंगा",u"ातीं",u"नाओं",u"नाएं",u"ताओं",u"ताएं",u"ियाँ",u"ियों",u"ियां"],
        5: [u"ाएंगी",u"ाएंगे",u"ाऊंगी",u"ाऊंगा",u"ाइयाँ",u"ाइयों",u"ाइयां"],
    }
    
    for L in suffixes:
        if len(word) > L + buffer:
            for idx,suf in enumerate(suffixes[L]):
                #print type(suf),type(word),word,suf
                if word.endswith(suf):
                    #print 'h'
                    return hi_to_en_map[word[:-L]]+suffixes_map[L][idx]
    return hi_to_en_map[word]

In [37]:
stories={}
for idx in tqdm(stories_idx):
    hi_story=hin_stories[idx]
    htokens=[stem_tokenize(x.text) for x in tok_hi(hi_story)]
    stories[idx]=" ".join(htokens)

100%|██████████| 19314/19314 [00:30<00:00, 630.03it/s]


In [38]:
stage4

{'आचंभित': 'aachambhit',
 'क्लोज': 'close',
 'ख़ुशहाल': 'khushahal',
 'गिराएंग': 'girayeng',
 'दुर्लभ,&#39;&#39': '',
 'वर्षीय': 'varshiya',
 'तत्पर': 'tatpar',
 'मिलत': 'milat',
 'लक्षण': 'lakshan',
 'वफ़ादार': 'waphadar',
 'पतंगबाज': 'patangabaaj',
 'संदर्भ': 'sandarbh',
 'नहलाय': 'nahlaye',
 'बुढ़िय': 'budhiya',
 'मौन': 'maun',
 'उगाएग': 'ugaeg',
 'कुण्ड': 'kund',
 'हास्य': 'hasya',
 'फ़्लैट': 'flat',
 'बाईसव': 'bicew',
 'गुर्राहट': 'gurrahat',
 'अड़सठ': 'adasat',
 'आइय': 'aay',
 'ज़ुज': 'zuz',
 'हेइद': 'heid',
 'इच्छानुसार': 'ichchanusar',
 'खाएँग': 'khaeng',
 'छुपात': 'chhupat',
 'परोसूँग': 'parosung',
 'लेग': 'leg',
 'कानाफूस': 'canafous',
 'माणिकत': 'manikat',
 'विख्यात': 'vikhyat',
 'खिंचन': 'khinchan',
 'सूँड़': 'sud',
 'गुलजार': 'gulzar',
 'ब्राज़ील': 'brazil',
 'क्राउन': 'crown',
 'हेलोवीन': 'helovin',
 'ख़ूब': 'khub',
 'टांग': 'tang',
 'डिज्ज': 'disge',
 'उड़ाएँ': 'udaen',
 'फिरत': 'firat',
 'नाचूंग': 'nachung',
 'सुनाऊं': 'sunaun',
 'गिनेंग': 'gineng',
 'जानन': 'janan',
 '

In [39]:
with open("hinglish_stories.json","w") as f:
    f.write(json.dumps(stories,indent=4))

In [40]:
len(stories)

19314

In [ ]:
count=0
shard={}
for k in stories:
    shard[k]=stories[k]
    count+=1
    if count%1000==0:
        with open(f"hinglish_stories_{count//1000}.json","w") as f:
            f.write(json.dumps(shard,indent=4))
        shard={}
with open(f"hinglish_stories_{count//1000+1}.json","w") as f:
    f.write(json.dumps(shard,indent=4))